In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pymongo
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import re

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["CathayDB"]
mycol = mydb["Cathay_591_data"]

In [2]:
url = 'https://rent.591.com.tw/?kind=0&region=1'

In [3]:
driver = webdriver.Chrome('/Users/johntung/Documents/workspace/CathayBK/Q2_591/chromedriver')
driver.maximize_window()
driver.get(url)
time.sleep(3)

driver.find_element_by_xpath('//*[@id="area-box-body"]/dl[1]/dd[1]').click()

In [4]:
while 'pageNext last' not in driver.page_source:
    #滑到底部
    js="var q=document.documentElement.scrollTop=100000"
    driver.execute_script(js)  
    time.sleep(5)
    
    next_page_button = driver.find_element_by_class_name('pageNext').find_element_by_tag_name('span')
    
    res = driver.page_source
    soup = BeautifulSoup(res, 'html.parser')
    
    for i in soup.find_all(class_ = 'listInfo clearfix j-house'):
        title = i.find('h3').find(target='_blank').text.strip()
        title_url = 'https:'+ i.find('h3').find('a')['href'].strip()
        renter = i.find_all('p')[2].find('em').text.split(' ')[1]
        identity = i.find_all('p')[2].find('em').text.split(' ')[0]
        house_situation = i.find(class_='lightBox').text.split('|')[0].strip()

        # 詳細資料進到網址爬取
        house_res = requests.get(title_url).text
        house_soup = BeautifulSoup(house_res, 'html.parser')

        # 因為某些物件沒有標示型態
        try:
            house_type = house_soup.find(class_='attr').find(lambda tag:tag.name=='li' and '型態' in tag.text).text.split(':')[1].strip()
        except:
            house_type = None

        phone = house_soup.find(class_='dialPhoneNum')['data-value']
        if phone == '':
            phone = house_soup.find(class_='hidtel').text

        
        gender = None
        for i in house_soup.find(class_='clearfix labelList labelList-1').find_all(class_='clearfix'):
            if '性' in i.find(class_='one').text:
                gender = i.find(class_='two').text.replace('：','')
                break
        
        insert_dict = {'renter':renter, 'identity':identity, 'phone':phone,
                       'house_type':house_type, 'house_situation':house_situation,'gender':gender,
                       'title':title, 'title_url':title_url, 'city':'台北市'}
        
#         print(title, title_url, renter, identity, house_type, phone, house_situation, gender)
        
        mycol.insert_one(insert_dict)
        print(insert_dict)
    
    next_page_button.click()
    print('next_page')

{'renter': '陳先生', 'identity': '屋主', 'phone': '0935-609-886', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '南京東六段/松山站/彩虹橋/大三房車位', 'title_url': 'https://rent.591.com.tw/rent-detail-10891802.html', 'city': '台北市', '_id': ObjectId('60a8b6ed6aa2fbcaf1248253')}
{'renter': '吳媽媽', 'identity': '屋主', 'phone': '0960-520-018', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '女生', 'title': '民生東路三段近松山機場,南京敦化精緻套房', 'title_url': 'https://rent.591.com.tw/rent-detail-10859679.html', 'city': '台北市', '_id': ObjectId('60a8b6ee6aa2fbcaf1248254')}
{'renter': '王小姐', 'identity': '屋主', 'phone': '0932-393-354', 'house_type': '公寓', 'house_situation': '整層住家', 'gender': None, 'title': '出入獨立一房一廳一衛浴個人設備', 'title_url': 'https://rent.591.com.tw/rent-detail-10936540.html', 'city': '台北市', '_id': ObjectId('60a8b6ef6aa2fbcaf1248255')}
{'renter': '陳先生', 'identity': '屋主', 'phone': '02-25569419', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '台北車站【五星飯店式】獨棟電梯~獨

{'renter': '鄧先生', 'identity': '屋主', 'phone': '0986-516-806', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '農安街電梯華夏整層出租（套房雅房分租', 'title_url': 'https://rent.591.com.tw/rent-detail-10930268.html', 'city': '台北市', '_id': ObjectId('60a8b6fe6aa2fbcaf1248270')}
next_page
{'renter': '蔡小姐', 'identity': '屋主', 'phone': '0906-698-613', 'house_type': '公寓', 'house_situation': '雅房', 'gender': '女生', 'title': '古亭捷運站【牯嶺公園旁】五星級雅房～限女', 'title_url': 'https://rent.591.com.tw/rent-detail-10731266.html', 'city': '台北市', '_id': ObjectId('60a8b7066aa2fbcaf1248271')}
{'renter': '華姊', 'identity': '屋主', 'phone': '0978-172-672', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': 'English萬隆捷運,近公館1房+廳', 'title_url': 'https://rent.591.com.tw/rent-detail-10928886.html', 'city': '台北市', '_id': ObjectId('60a8b70c6aa2fbcaf1248272')}
{'renter': '李小姐', 'identity': '屋主', 'phone': '0955-860-558', 'house_type': '公寓', 'house_situation': '雅房', 'gender': '男女生皆可', 'title': '台大北教

{'renter': '江太太', 'identity': '屋主', 'phone': '0911-684-994', 'house_type': '電梯大樓', 'house_situation': '分租套房', 'gender': '女生', 'title': '全新有陽台有廚房(獨立電錶)(近捷運站)', 'title_url': 'https://rent.591.com.tw/rent-detail-10825686.html', 'city': '台北市', '_id': ObjectId('60a8b71d6aa2fbcaf124828d')}
{'renter': '蔡先生', 'identity': '屋主', 'phone': '0981-051-005', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '✅六星✅近中山國中捷運✅近公園', 'title_url': 'https://rent.591.com.tw/rent-detail-10890781.html', 'city': '台北市', '_id': ObjectId('60a8b71d6aa2fbcaf124828e')}
next_page
{'renter': '葉先生', 'identity': '屋主', 'phone': '0920-416-056', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '榮星高樓挑高2房含車位', 'title_url': 'https://rent.591.com.tw/rent-detail-10861311.html', 'city': '台北市', '_id': ObjectId('60a8b7246aa2fbcaf124828f')}
{'renter': '何先生', 'identity': '屋主', 'phone': '0928-619-901', 'house_type': '透天厝', 'house_situation': '其他', 'gender': None, 'title': '景色優美透天出租', 'tit

{'renter': '林小姐', 'identity': '屋主', 'phone': '0933-919-593', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '敦南遠企安和林蔭3房平面車位有收垃圾', 'title_url': 'https://rent.591.com.tw/rent-detail-10907577.html', 'city': '台北市', '_id': ObjectId('60a8b7346aa2fbcaf12482aa')}
{'renter': '陳小姐', 'identity': '仲介', 'phone': '0939-987-096', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 'title': '【獨家專任】台北101世貿捷運精緻2房', 'title_url': 'https://rent.591.com.tw/rent-detail-10870631.html', 'city': '台北市', '_id': ObjectId('60a8b7346aa2fbcaf12482ab')}
{'renter': '古先生', 'identity': '屋主', 'phone': '0935-733-270', 'house_type': '別墅', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '中國文化大學的學生步行上課。可容納4個', 'title_url': 'https://rent.591.com.tw/rent-detail-10926325.html', 'city': '台北市', '_id': ObjectId('60a8b7356aa2fbcaf12482ac')}
next_page
{'renter': '曾先生', 'identity': '屋主', 'phone': '0986-851-077 轉 1471671', 'house_type': '電梯大樓', 'house_situation': '整層住家', 'gender': None, 't

{'renter': '吳小姐', 'identity': '屋主', 'phone': '0936-884-980', 'house_type': '電梯大樓', 'house_situation': '雅房', 'gender': '女生', 'title': '公館、台電捷運全新雅房', 'title_url': 'https://rent.591.com.tw/rent-detail-10810756.html', 'city': '台北市', '_id': ObjectId('60a8b7516aa2fbcaf12482c7')}
{'renter': '林小姐', 'identity': '屋主', 'phone': '0935-596-961', 'house_type': '公寓', 'house_situation': '整層住家', 'gender': None, 'title': '幽雅4房樓中樓(含平面車位)', 'title_url': 'https://rent.591.com.tw/rent-detail-10889299.html', 'city': '台北市', '_id': ObjectId('60a8b7526aa2fbcaf12482c8')}
{'renter': '曾小姐', 'identity': '屋主', 'phone': '0906-288-237', 'house_type': '公寓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '西湖站/劍潭站.獨立陽台.溫馨設計感套房', 'title_url': 'https://rent.591.com.tw/rent-detail-10923356.html', 'city': '台北市', '_id': ObjectId('60a8b7526aa2fbcaf12482c9')}
{'renter': '高先生', 'identity': '屋主', 'phone': '0935-068-711', 'house_type': '透天厝', 'house_situation': '雅房', 'gender': '女生', 'title': '景美雅房廉租近景美捷運站單人房適世新學生', 'title_

...

{'renter': '施小姐', 'identity': '屋主', 'phone': '0920-771-687', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '辛亥路5段近捷運萬芳醫院、辛亥站中國科大', 'title_url': 'https://rent.591.com.tw/rent-detail-10752427.html', 'city': '台北市', '_id': ObjectId('60a8d8406aa2fbcaf124aa37')}
{'renter': '羅先生', 'identity': '代理人', 'phone': '02-27058016', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '地下B1室內平面車位', 'title_url': 'https://rent.591.com.tw/rent-detail-10732243.html', 'city': '台北市', '_id': ObjectId('60a8d8406aa2fbcaf124aa38')}
{'renter': '黃先生', 'identity': '屋主', 'phone': '0986-851-077 轉 1432155', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '敦化國小、國中雙學區出租設學籍！', 'title_url': 'https://rent.591.com.tw/rent-detail-10710567.html', 'city': '台北市', '_id': ObjectId('60a8d8416aa2fbcaf124aa39')}
next_page
{'renter': '施小姐', 'identity': '屋主', 'phone': '0920-771-687', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '辛亥

{'renter': '黃先生', 'identity': '屋主', 'phone': '0986-851-077 轉 1432155', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '敦化國小、國中雙學區出租設學籍！', 'title_url': 'https://rent.591.com.tw/rent-detail-10710567.html', 'city': '台北市', '_id': ObjectId('60a8d8826aa2fbcaf124aa54')}
next_page
{'renter': '施小姐', 'identity': '屋主', 'phone': '0920-771-687', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '辛亥路5段近捷運萬芳醫院、辛亥站中國科大', 'title_url': 'https://rent.591.com.tw/rent-detail-10752427.html', 'city': '台北市', '_id': ObjectId('60a8d8886aa2fbcaf124aa55')}
{'renter': '羅先生', 'identity': '代理人', 'phone': '02-27058016', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '地下B1室內平面車位', 'title_url': 'https://rent.591.com.tw/rent-detail-10732243.html', 'city': '台北市', '_id': ObjectId('60a8d8886aa2fbcaf124aa56')}
{'renter': '黃先生', 'identity': '屋主', 'phone': '0986-851-077 轉 1432155', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 

{'renter': '羅先生', 'identity': '代理人', 'phone': '02-27058016', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '地下B1室內平面車位', 'title_url': 'https://rent.591.com.tw/rent-detail-10732243.html', 'city': '台北市', '_id': ObjectId('60a8d8c76aa2fbcaf124aa71')}
{'renter': '黃先生', 'identity': '屋主', 'phone': '0986-851-077 轉 1432155', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '敦化國小、國中雙學區出租設學籍！', 'title_url': 'https://rent.591.com.tw/rent-detail-10710567.html', 'city': '台北市', '_id': ObjectId('60a8d8c76aa2fbcaf124aa72')}
next_page
{'renter': '施小姐', 'identity': '屋主', 'phone': '0920-771-687', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '辛亥路5段近捷運萬芳醫院、辛亥站中國科大', 'title_url': 'https://rent.591.com.tw/rent-detail-10752427.html', 'city': '台北市', '_id': ObjectId('60a8d8cd6aa2fbcaf124aa73')}
{'renter': '羅先生', 'identity': '代理人', 'phone': '02-27058016', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '地下B1室內平

{'renter': '施小姐', 'identity': '屋主', 'phone': '0920-771-687', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '辛亥路5段近捷運萬芳醫院、辛亥站中國科大', 'title_url': 'https://rent.591.com.tw/rent-detail-10752427.html', 'city': '台北市', '_id': ObjectId('60a8d90b6aa2fbcaf124aa8e')}
{'renter': '羅先生', 'identity': '代理人', 'phone': '02-27058016', 'house_type': None, 'house_situation': '車位', 'gender': None, 'title': '地下B1室內平面車位', 'title_url': 'https://rent.591.com.tw/rent-detail-10732243.html', 'city': '台北市', '_id': ObjectId('60a8d90c6aa2fbcaf124aa8f')}
{'renter': '黃先生', 'identity': '屋主', 'phone': '0986-851-077 轉 1432155', 'house_type': '電梯大樓', 'house_situation': '獨立套房', 'gender': '男女生皆可', 'title': '敦化國小、國中雙學區出租設學籍！', 'title_url': 'https://rent.591.com.tw/rent-detail-10710567.html', 'city': '台北市', '_id': ObjectId('60a8d90c6aa2fbcaf124aa90')}
next_page
{'renter': '施小姐', 'identity': '屋主', 'phone': '0920-771-687', 'house_type': '公寓', 'house_situation': '分租套房', 'gender': '男女生皆可', 'title': '辛亥

KeyboardInterrupt: 